In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [2]:
#importing libraries 
import sklearn
import numpy as np
from glob import glob
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline 

import string
from os import listdir
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from pickle import dump
from string import punctuation

from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers import LSTM
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from pickle import load

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

import csv, string

Using TensorFlow backend.


In [3]:
categories = ['Gender abusive hate', 'Geopoitical Hate','Religious Hate','Political Normal','Political Hate','Personal Hate']
num_of_labels=len(categories)
categories_dict={key:value for value,key in enumerate(categories)}
categories_inverse_dict={key:value for key,value in enumerate(categories)}
print(categories_dict)

{'Gender abusive hate': 0, 'Geopoitical Hate': 1, 'Religious Hate': 2, 'Political Normal': 3, 'Political Hate': 4, 'Personal Hate': 5}


In [4]:
stop_words='../stopwords-bn.txt'
text_data=[]
with open(stop_words,'r',encoding='utf-8') as temp_output_file:
    reader=csv.reader(temp_output_file, delimiter='\n')
    for row in reader:
        text_data.append(row)
stop_word_list=[x[0] for x in text_data]
print(stop_word_list[0:2])
print(type(stop_word_list))

['অতএব', 'অথচ']
<class 'list'>


In [5]:
def save_dataset(dataset, filename):
    dump(dataset, open(filename,'wb'))
    print('Saved :%s' % filename)

def load_doc(filename):
    file=open(filename,'r',encoding='utf-8')
    text=file.read()
    file.close()
    return text


def clean_doc(doc, stop_word_list):
    # split into tokens by white space
    sentences=list()
    
    for sentence in doc:
        # remove punctuation from each token
        
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in sentence.split(' ')]  # I belive word in sentence
        sentences.append(' '.join(tokens))
    return sentences



In [6]:
def load_data_and_labels(filename,stop_word_list):
    doc=load_doc(filename)
    x_doc=list()
    label=list()
    for line in doc.split('\n'):
        x_and_label=line.split('\t')
        #print(x_and_label)
        x_doc.append(x_and_label[1])
        label.append(categories_dict[x_and_label[0]])
        
    #print(x_doc)
    #print(label)
    trainX=clean_doc(x_doc, stopwords)
    return trainX, label
    

In [15]:
trainX,trainY=load_data_and_labels('../Hate_Speech_Complete.csv',stop_word_list)

In [16]:
print(trainX[0:10])

['এদের কারনে এলিয়েনরা পৃথিবীতে বেড়াতে আসে না।', 'এই গান গুলা হুইন্না আমি এখন আমি পাবনা তে আসি', 'ভাই ছোট আজাদ চুদনাপানা না করলেই কি না ', 'সব মাদার চোদ। সালারা তোদের কাছ থেকে কি শিখব ', 'বাইরে থাক আর গুঁতাগুঁতি কর ', 'ইংলিশ এ Wiz khalifa Weed টানে আর বাঙালিরারা টানলেই দোষ ধইরা ফেল্লা', 'আসোলে তুমি নিজেই বলসিলা বাংগালি না আমরা। জাতটাই ত খারাপ', 'আর আমাদের এই খাইস্টা জাতের মধ্যে তুমিও পড় সো রক্ত ত মিশশা আসেই জিনিস টা', 'শালা তুই সালমান রিদিয়া আসিফসব পাকনাপাকনা পুলাপাইন ', 'এসব আবাল গুলা আমাদের সুন্দর বিনোদন থেকে দূরে রাখে। ']


In [17]:
def create_tokenizer(lines):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def encode_text(tokenizer, lines, length):
    #print(lines)
    encoded=tokenizer.texts_to_sequences(lines)
    print(encoded[0:10])
    padded= pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [18]:
tokenizer=create_tokenizer(trainX)

In [19]:
trainLength=300
vocab_size=len(tokenizer.word_index)+1
print('Max document length: %d' % trainLength)
print('Vocabulary size: %d' % vocab_size)
print()
trainX = encode_text(tokenizer, trainX, trainLength)
print(trainX.shape)
#trainX[2]

Max document length: 300
Vocabulary size: 5034

[[194, 339, 1537, 636, 1538, 637, 70], [2, 340, 195, 1539, 33, 52, 33, 1540, 341, 908], [21, 221, 253, 1541, 1, 1542, 6, 1], [12, 111, 1543, 487, 34, 1544, 28, 6, 1545], [408, 909, 5, 1546, 179], [254, 66, 1547, 1548, 1549, 1550, 5, 1551, 1552, 910, 911, 1553], [1554, 222, 912, 1555, 1556, 1, 1557, 1558, 255, 112], [5, 23, 2, 1559, 1560, 151, 1561, 1562, 488, 489, 255, 1563, 1564, 1565, 53], [39, 7, 913, 1566, 1567, 1568, 914], [180, 162, 195, 23, 256, 1569, 28, 1570, 1571]]
(1400, 300)


In [20]:
NUM_WORDS=20000
word_vectors = Word2Vec.load('../posts.bin')
EMBEDDING_DIM=300
vocabulary_size=len(tokenizer.word_index)+1
word_index=tokenizer.word_index
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector=word_vectors[word]
        embedding_matrix[i]=embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

/home/bharaj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [21]:
embedding_layer=Embedding(vocabulary_size, EMBEDDING_DIM, weights=[embedding_matrix],trainable=True)

In [35]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	input1 = Input(shape=(length,))
	embedding_layer_1 = embedding_layer(input1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding_layer_1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
    
	# channel 2
	input2 = Input(shape=(length,))
	embedding_layer_2 = embedding_layer(input2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding_layer_2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
    
	# channel 3
	input3 = Input(shape=(length,))
	embedding_layer_3 = embedding_layer(input3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding_layer_3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
    
	# merge
	CNN_layer = concatenate([flat1, flat2, flat3])
    
	# LSTM
	x = embedding_layer(input3)
	LSTM_layer = LSTM(128)(x)

	CNN_LSTM_layer = concatenate([LSTM_layer, CNN_layer])
    
	# interpretation
	dense1 = Dense(10, activation='relu')(CNN_LSTM_layer)
	outputs = Dense(num_of_labels, activation='softmax')(dense1)
	model = Model(inputs=[input1, input2, input3], outputs=outputs)
    
	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
	# summarize
	print(model.summary())
    
	return model

In [36]:
model= define_model(300,5034)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     1510200     input_10[0][0]                   
                                                                 input_11[0][0]                   
          

In [37]:
from keras.utils import to_categorical
y_binary = to_categorical(trainY)
print(y_binary)

[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [39]:
model.fit([trainX,trainX,trainX], array(y_binary), epochs=1, batch_size=128, validation_split=0.2)

Train on 1120 samples, validate on 280 samples
Epoch 1/1
1120/1120 [==============================] - 9s 8ms/step - loss: 1.7872 - acc: 0.3571 - val_loss: 1.8007 - val_acc: 0.0000e+00
